In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive

In [5]:
url = "https://drive.google.com/drive/folders/15dFAVS_3xAgxLSq12DZjODVIc10nxvrA?usp=drive_link"

### SE CONNECTER A GOOGLE DRIVE/ ENVIRONNEMENT DE TRAVAIL GOOGLE COLAB

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [20]:
dataset_directory = '/content/drive/My Drive/images/train'

### FAIRE DE LA DATA AUGMENTATION EN UTILISANT IMAGEDATAGENERATOR

In [1]:
datagen = ImageDataGenerator(
    rescale=1./255,       
    shear_range=0.2,      
    zoom_range=0.2,       
    horizontal_flip=True 
)


NameError: name 'ImageDataGenerator' is not defined

In [16]:
batch_size = 32
train_generator = datagen.flow_from_directory(
    dataset_directory ,  
    target_size=(224, 224),        
    batch_size=batch_size,
    class_mode='categorical'            
)

Found 261 images belonging to 7 classes.


### CREATION DES LAYERS AVEC UN TRANSFER LEARNING ( RESNET 50)

In [17]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Creation de la base de resnet50
base_model = ResNet50(weights='imagenet', include_top=False)
num_classes = 7

x = base_model.output
x = GlobalAveragePooling2D()(x)

# ici on ajoute le fully connected layers
x = Dense(1024, activation='relu')(x)

# Softmax pour la classification multiclasse
predictions = Dense(num_classes, activation='softmax')(x)

# Model final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrainement du model avec data generator
model.fit(train_generator, epochs=3, steps_per_epoch=len(train_generator), verbose=1)


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/3
9/9 [==============================] - 204s 19s/step - loss: 2.0461 - accuracy: 0.6820
Epoch 2/3
9/9 [==============================] - 176s 22s/step - loss: 0.9378 - accuracy: 0.7701
Epoch 3/3
9/9 [==============================] - 176s 19s/step - loss: 0.8858 - accuracy: 0.7701


### AUTRES FACONS C'EST DE CREER UN MODEL COMME RESNET FROM SCRATCH

In [18]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Flatten, Dense
from tensorflow.keras.models import Model

def identity_block(X, f, filters):
    F1, F2, F3 = filters
    #ICI tous les lower layer avec les fonctions d'activation
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, s=2):
    F1, F2, F3 = filters

    X_shortcut = X
    #ICI LES BLOCKS FONCTIONNEL DE KERAS / PAS DE SEQUENCE
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape=(48, 48, 3), classes=1000):
    X_input = Input(input_shape)

    X = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    #IL FAUT APPLATIR LES DONNEES POUR UNE ACTIVATION MULTICALL(SOFTMAX)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X)

    return model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=3, steps_per_epoch=len(train_generator), verbose=1)


Epoch 1/3
9/9 [==============================] - 200s 19s/step - loss: 1.3642 - accuracy: 0.7471
Epoch 2/3
9/9 [==============================] - 177s 19s/step - loss: 1.1194 - accuracy: 0.7625
Epoch 3/3
9/9 [==============================] - 178s 19s/step - loss: 1.0384 - accuracy: 0.7701


### Un model plus personnalisé sur le même concept que resnet50

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Flatten, Dense
from tensorflow.keras.models import Model

class IdentityBlock(tf.keras.layers.Layer):
    def __init__(self, filters, f, **kwargs):
        super(IdentityBlock, self).__init__(**kwargs)
        self.F1, self.F2, self.F3 = filters
        self.f = f

    def build(self, input_shape):
        self.conv1 = Conv2D(filters=self.F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')
        self.bn1 = BatchNormalization(axis=3)
        self.act1 = Activation('relu')

        self.conv2 = Conv2D(filters=self.F2, kernel_size=(self.f, self.f), strides=(1, 1), padding='same')
        self.bn2 = BatchNormalization(axis=3)
        self.act2 = Activation('relu')

        self.conv3 = Conv2D(filters=self.F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')
        self.bn3 = BatchNormalization(axis=3)

        self.add = Add()
        self.act3 = Activation('relu')

    def call(self, inputs):
        X_shortcut = inputs

        X = self.conv1(inputs)
        X = self.bn1(X)
        X = self.act1(X)

        X = self.conv2(X)
        X = self.bn2(X)
        X = self.act2(X)

        X = self.conv3(X)
        X = self.bn3(X)

        X = self.add([X, X_shortcut])
        X = self.act3(X)

        return X

class ConvolutionalBlock(tf.keras.layers.Layer):
    def __init__(self, filters, f, s=2, **kwargs):
        super(ConvolutionalBlock, self).__init__(**kwargs)
        self.F1, self.F2, self.F3 = filters
        self.f = f
        self.s = s

    def build(self, input_shape):
        self.conv1 = Conv2D(filters=self.F1, kernel_size=(1, 1), strides=(self.s, self.s), padding='valid')
        self.bn1 = BatchNormalization(axis=3)
        self.act1 = Activation('relu')

        self.conv2 = Conv2D(filters=self.F2, kernel_size=(self.f, self.f), strides=(1, 1), padding='same')
        self.bn2 = BatchNormalization(axis=3)
        self.act2 = Activation('relu')

        self.conv3 = Conv2D(filters=self.F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')
        self.bn3 = BatchNormalization(axis=3)

        self.conv_shortcut = Conv2D(filters=self.F3, kernel_size=(1, 1), strides=(self.s, self.s), padding='valid')
        self.bn_shortcut = BatchNormalization(axis=3)

        self.add = Add()
        self.act3 = Activation('relu')

    def call(self, inputs):
        X_shortcut = inputs

        X = self.conv1(inputs)
        X = self.bn1(X)
        X = self.act1(X)

        X = self.conv2(X)
        X = self.bn2(X)
        X = self.act2(X)

        X = self.conv3(X)
        X = self.bn3(X)

        X_shortcut = self.conv_shortcut(X_shortcut)
        X_shortcut = self.bn_shortcut(X_shortcut)

        X = self.add([X, X_shortcut])
        X = self.act3(X)

        return X

class ResNet48(tf.keras.Model):
    def __init__(self, classes=7, **kwargs):
        super(ResNet48, self).__init__(**kwargs)
        self.classes = classes

        self.conv1 = Conv2D(64, (7, 7), strides=(2, 2), padding='same')
        self.bn1 = BatchNormalization(axis=3)
        self.act1 = Activation('relu')
        self.maxpool1 = MaxPooling2D((3, 3), strides=(2, 2))

        self.conv_block1 = ConvolutionalBlock([64, 64, 256], 3, s=1)
        self.id_block1 = IdentityBlock([64, 64, 256], 3)
        self.id_block2 = IdentityBlock([64, 64, 256], 3)

        self.conv_block2 = ConvolutionalBlock([128, 128, 512], 3, s=2)
        self.id_block3 = IdentityBlock([128, 128, 512], 3)
        self.id_block4 = IdentityBlock([128, 128, 512], 3)
        self.id_block5 = IdentityBlock([128, 128, 512], 3)

        self.conv_block3 = ConvolutionalBlock([256, 256, 1024], 3, s=2)
        self.id_block6 = IdentityBlock([256, 256, 1024], 3)
        self.id_block7 = IdentityBlock([256, 256, 1024], 3)
        self.id_block8 = IdentityBlock([256, 256, 1024], 3)
        self.id_block9 = IdentityBlock([256, 256, 1024], 3)
        self.id_block10 = IdentityBlock([256, 256, 1024], 3)

        self.conv_block4 = ConvolutionalBlock([512, 512, 2048], 3, s=2)
        self.id_block11 = IdentityBlock([512, 512, 2048], 3)
        self.id_block12 = IdentityBlock([512, 512, 2048], 3)

        self.flatten = Flatten()
        self.dense = Dense(classes, activation='softmax')

    def call(self, inputs):
        X = self.conv1(inputs)
        X = self.bn1(X)
        X = self.act1(X)
        X = self.maxpool1(X)

        X = self.conv_block1(X)
        X = self.id_block1(X)
        X = self.id_block2(X)

        X = self.conv_block2(X)
        X = self.id_block3(X)
        X = self.id_block4(X)
        X = self.id_block5(X)

        X = self.conv_block3(X)
        X = self.id_block6(X)
        X = self.id_block7(X)
        X = self.id_block8(X)
        X = self.id_block9(X)
        X = self.id_block10(X)

        X = self.conv_block4(X)
        X = self.id_block11(X)
        X = self
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=3, steps_per_epoch=len(train_generator), verbose=1)

Epoch 1/3
9/9 [==============================] - 200s 19s/step - loss: 1.0549 - accuracy: 0.7739
Epoch 2/3
9/9 [==============================] - 176s 21s/step - loss: 0.9771 - accuracy: 0.7701
Epoch 3/3
9/9 [==============================] - 176s 19s/step - loss: 1.2134 - accuracy: 0.7701


In [21]:
test_directory = '/content/drive/My Drive/images/validation'

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the test images
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Loss: {loss}, Accuracy: {accuracy}')

Found 70 images belonging to 7 classes.
3/3 [==============================] - 12s 5s/step - loss: 14166.0811 - accuracy: 0.1429
Loss: 14166.0810546875, Accuracy: 0.1428571492433548


### ENVOYER L'API SUR GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

app = Flask(__name__)

# Load the model
model = load_model('/content/drive/My Drive/images/model.h5')